### Import lib and extract zip

In [1]:
import sklearn
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
#from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install emot
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

     |████████████████████████████████| 61 kB 18 kB/s 


In [ ]:
# print working directory
!pwd

/content


In [2]:
# change current directory
%cd /content/drive/My\ Drive/Udemy Courses/Natural Language Processing: Machine Learning NLP In Python/16.Create A Web App to Deploy

/content/drive/My Drive/Udemy Courses/Natural Language Processing: Machine Learning NLP In Python/16.Create A Web App to Deploy


In [ ]:
!unzip trainingandtestdata.zip

Archive:  trainingandtestdata.zip
  inflating: testdata.manual.2009.06.14.csv  
  inflating: training.1600000.processed.noemoticon.csv  


### Read Data

In [ ]:
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin-1")
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


#### Adding columns to dataframe

In [ ]:
cols = ['sentiment', 'id', 'date', 'query_string', 'user', 'text']
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin-1", names=cols)
df.head()

,sentiment,id,date,query_string,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


### Preprocessing

In [ ]:
def hashtags(text):
  hash = re.findall(r"#(\w+)", text)
  return hash

def emoji(text):
  for emot in UNICODE_EMOJI:
    if text == None:
      text = text
    else:
      text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",", "").replace(":", "").split()))
    return text

def remove_users(tweet):
  '''Takes a string and removes retweet and @user information'''
  tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
  # remove tweeted at
  return tweet

def remove_links(tweet):
  '''Takes a string and removes web links from it'''
  tweet = re.sub(r'http\S+', '', tweet) # remove http links
  tweet = re.sub(r'bit.ly/\S+', '', tweet) # remove bitly links
  tweet = tweet.strip('[link]') # remove [links]
  return tweet

def clean_html(text):
  html = re.compile('<.*?>')#regex
  return html.sub(r'',text)

def non_ascii(s):
  return "".join(i for i in s if ord(i)<128)

def lower(text):
  return text.lower()

def removeStopWords(str):
  #select english stopwords
  cachedStopWords = set(stopwords.words("english"))
  #add custom words
  cachedStopWords.update(('and','I','A','http','And','So','arnt','This','When','It','many','Many','so','cant','Yes','yes','No','no','These','these','mailto','regards','ayanna','like','email'))
  #remove stop words
  new_str = ' '.join([word for word in str.split() if word not in cachedStopWords]) 
  return new_str

def email_address(text):
  email = re.compile(r'[\w\.-]+@[\w\.-]+')
  return email.sub(r'',text)

def punct(text):
  token=RegexpTokenizer(r'\w+')#regex
  text = token.tokenize(text)
  text= " ".join(text)
  return text 

def remove_digits(text):
  pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
  #when the ^ is on the inside of []; we are matching any character that is not included in this expression within the []
  return re.sub(pattern, '', text)

def remove_special_characters(text):
  # define the pattern to keep
  pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
  return re.sub(pat, '', text)

def remove_(tweet):
  tweet = re.sub('([_]+)', "", tweet)
  return tweet


In [ ]:
df['hashtag'] = df.text.apply(func = hashtags)
df['new_tweet'] = df.text.apply(func = emoji)
df['new_tweet'] = df.new_tweet.apply(func = remove_users)
df['new_tweet'] = df.new_tweet.apply(func = clean_html)
df['new_tweet'] = df.new_tweet.apply(func = non_ascii)
df['new_tweet'] = df.new_tweet.apply(func = lower)
df['new_tweet'] = df.new_tweet.apply(func = email_address)
df['new_tweet'] = df.new_tweet.apply(func = removeStopWords)
df['new_tweet'] = df.new_tweet.apply(func = clean_html)
df['new_tweet'] = df.new_tweet.apply(func = punct)
df['new_tweet'] = df.new_tweet.apply(func = remove_)

In [ ]:
df.tail()

,sentiment,id,date,query_string,user,text,hashtag,new_tweet
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,[],woke up school best feeling ever
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,[],thewdb com cool hear old walt interviews http ...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,[],ready mojo makeover ask details
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,[],happy 38th birthday boo alll time tupac amaru ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...,[charitytuesday],happy charitytuesday


In [ ]:
def clean(raw):
  result = re.sub('<[a][^>]*>(.+?)</[a]>', 'Link.', raw)
  result = re.sub('&gt;', "", result)
  result = re.sub('&#x27;', "'", result)
  result = re.sub('&quot;', '"', result)
  result = re.sub('&#x2F;', ' ', result)
  result = re.sub('<p>', ' ', result)
  result = re.sub('</i>', '', result)
  result = re.sub('&#62;', '', result)
  result = re.sub('<i>', ' ', result)
  result = re.sub("\n", '', result)
  return result

In [ ]:
df['new_tweet2'] = df.text.apply(func = clean)

In [ ]:
df.tail()

,sentiment,id,date,query_string,user,text,hashtag,new_tweet,new_tweet2
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,[],woke up school best feeling ever,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,[],thewdb com cool hear old walt interviews http ...,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,[],ready mojo makeover ask details,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,[],happy 38th birthday boo alll time tupac amaru ...,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...,[charitytuesday],happy charitytuesday,happy #charitytuesday @theNSPCC @SparksCharity...


### Split Dataset 

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.3, stratify=df['sentiment'], random_state=21)
print(df_train.shape, df_test.shape)

(1120000, 9) (480000, 9)


### TF-IDF
Create a TF-IDF vector of the tweet text as defined above

In [ ]:
tfidf_vectorizer = TfidfVectorizer(lowercase=True, max_features=1000, stop_words=ENGLISH_STOP_WORDS)
tfidf_vectorizer.fit(df_train.new_tweet2)

TfidfVectorizer(max_features=1000,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}))

In [ ]:
train_idf = tfidf_vectorizer.transform(df_train.new_tweet2)
test_idf = tfidf_vectorizer.transform(df_test.new_tweet2)

### Random Forest Model
create the object of RF Model
fit the model with the training data
predict the label on the training data
predict the model on the test data
f1 score on train data

In [ ]:
model_rf = RandomForestClassifier(n_estimators=20)
model_rf.fit(train_idf, df_train.sentiment)
predict_train = model_rf.predict(train_idf)
predict_test = model_rf.predict(test_idf)

In [ ]:
print(sklearn.metrics.precision_score(y_true=df_train.sentiment, y_pred=predict_train, pos_label=4))
print(sklearn.metrics.precision_score(y_true=df_test.sentiment, y_pred=predict_test, pos_label=4))

0.9064657953011623
0.7256212789303639


### Pipeline

In [ ]:
pipeline = Pipeline(steps=[('tfidf', TfidfVectorizer(lowercase=True, max_features=1000, stop_words=ENGLISH_STOP_WORDS)),
                           ('model', RandomForestClassifier(n_estimators=40))])
pipeline.fit(df_train.new_tweet2, df_train.sentiment)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=frozenset({'a', 'about', 'above',
                                                       'across', 'after',
                                                       'afterwards', 'again',
                                                       'against', 'all',
                                                       'almost', 'alone',
                                                       'along', 'already',
                                                       'also', 'although',
                                                       'always', 'am', 'among',
                                                       'amongst', 'amoungst',
                                                       'amount', 'an', 'and',
                                                       'another', 'any',
                                                       'anyhow', 'anyone',
           

In [ ]:
txt = ["this restaurant is good, their service is preety awesome"]
pipeline.predict(txt)

array([4])

### Joblib
Used to save pipeline object

In [ ]:
from joblib import dump
dump(pipeline, filename="text_classification.joblib")

['text_classification.joblib']

In [12]:
from joblib import load
pipeline = load("text_classification.joblib")

In [ ]:
!pwd

/content


### Setup Twitter API

In [ ]:
!pip install tweepy

In [3]:
import tweepy
import time
pd.set_option('display.max_colwidth', 1000)

api_key = "QsVk97gXC6Y0ISWIk3Mn1jl7q"
api_secret_key = "mkna7mzVmboExLIG3azuyWI6NwjaUeyuIS15C1OkuEYNoagxWh"
access_token = "361125027-KS9CHs4dgsB5sfLdaBJWV53R6QulPRwSrqn5cVrO"
access_token_secret = "RnGLDhZBaxqtBkQMx9R30WUedkIHQW2OC1rVWl1j2HNtz"

auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

##### Function search tweet
This function will take parameter text_query and return 50 tweets related to that particular text query. 
We will use the search API to get the results from Twitter.

In [4]:
def get_related_tweets(text_query):
    # list to store tweets
    tweets_list = []
    # no of tweets
    count = 50
    try:
        # Pulling individual tweets from query
        for tweet in api.search(q=text_query, count=count):
            print(tweet.text)
            # Adding to list that contains all tweets
            tweets_list.append({'created_at': tweet.created_at,
                                'tweet_id': tweet.id,
                                'tweet_text': tweet.text})
        return pd.DataFrame.from_dict(tweets_list)

    except BaseException as e:
        print('failed on_status,', str(e))
        time.sleep(3)

In [7]:
from flask import Flask, render_template, request, redirect, url_for
from joblib import load
from flask_ngrok import run_with_ngrok

# load the pipeline object
pipeline = load("text_classification.joblib")

# function to get results for a particular text query
def requestResults(name):
    # get the tweets text
    tweets = get_related_tweets(name)
    # get the prediction
    tweets['prediction'] = pipeline.predict(tweets['tweet_text'])
    # get the value counts of different labels predicted
    data = str(tweets.prediction.value_counts()) + '\n\n'
    return data + str(tweets)

In [8]:
app = Flask(__name__)
run_with_ngrok(app)
# render default webpage
@app.route('/')
def home():
    return render_template('sentimentanalysis.html')

# when the post method detect, then redirect to success function
@app.route('/', methods=['POST', 'GET'])
def get_data():
    if request.method == 'POST':
        user = request.form['search']
        return redirect(url_for('success', name=user))

# get the data for the requested query
@app.route('/success/<name>')
def success(name):
    return "<xmp>" + str(requestResults(name)) + " </xmp> "

In [11]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://308d-35-238-246-95.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2022-01-08 11:40:20,565] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-8-809116b20fe5>", line 6, in home
    return render_template('sentimentanalysis.html')
  Fi